# Box Loader Testing

I need to test a few things about the steps in the `argo_box_loader` notebook. There are a few things that might make a difference by decreasing the size of an array, but I want to try here on a box that takes a lot less time to load.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw
import traceback
from importlib import reload

import argopy
from argopy import DataFetcher as ArgoDataFetcher

argo_loader = ArgoDataFetcher(
    src="gdac", ftp="/swot/SUM05/dbalwada/Argo_sync", progress=True
)

In [2]:
import os
import sys
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
sys.path.append(os.path.abspath('../funcs'))

import processing_funcs as prf

In [3]:
import dask
from dask.distributed import Client, LocalCluster

# Create a LocalCluster
cluster = LocalCluster(n_workers=32, threads_per_worker=2, memory_limit='30GiB')

# Connect to the cluster
client = Client(cluster)

# Print the cluster information
print(cluster)

/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42055 instead
  warnings.warn(


LocalCluster(bbba1d7e, 'tcp://127.0.0.1:35433', workers=32, threads=64, memory=0.94 TiB)


Task exception was never retrieved
future: <Task finished name='Task-65485' coro=<Client._gather.<locals>.wait() done, defined at /home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-65452' coro=<Client._gather.<locals>.wait() done, defined at /home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-65454' 

In [10]:
@dask.delayed
def get_box_delayed(*args, **kwargs):
    try:
        return prf.get_box(*args, **kwargs)
    except Exception as e:
        return type(e).__name__, str(e)

def get_box_dask(boxes_list):
    
    boxes_list = [tuple([box]) for box in boxes_list]
    
    # Create a list of delayed tasks
    tasks = [get_box_delayed(*args) for args in boxes_list]

    # Compute the results in parallel
    results = dask.compute(*tasks)
    
    errors=[]

    # Process the results as needed
    for n, result in enumerate(results):
        if isinstance(result, tuple) and isinstance(result[0], str):
            error_type, error_message = result
            print("Error in box {}: {} - {}".format([boxes_list[n][0][0],boxes_list[n][0][1],boxes_list[n][0][2],boxes_list[n][0][3]], error_type, error_message))
            errors.append([boxes_list[n][0][0],boxes_list[n][0][1],boxes_list[n][0][2],boxes_list[n][0][3], error_type, error_message])
        else:
            result.to_netcdf("/swot/SUM05/amf2288/sync-boxes/workflow_test/lon:({},{})_lat:({},{})_ds_z.nc".format(boxes_list[n][0][0],boxes_list[n][0][1],boxes_list[n][0][2],boxes_list[n][0][3]))
            print("Saved box {} of {}".format(n+1, len(results)))
            
    return errors 

In [11]:
from typing import List, Tuple

#def generate_grid(min_lat: float, max_lat: float, min_lon: float, max_lon: float, step: float) -> List[Tuple[float, float, float, float]]:
def generate_grid(box, step):
    grid = []
    lon_min, lon_max, lat_min, lat_max = box[0],box[1],box[2],box[3]
    lat = lat_min
    while lat < lat_max:
        lon = lon_min
        while lon < lon_max:
            # Ensure the box does not exceed the specified bounds
            box_lat_max = min(lat + step, lat_max)
            box_lon_max = min(lon + step, lon_max)
            box = [lat, box_lat_max, lon, box_lon_max, box[4], box[5]]
            grid.append(box)
            lon += step
        lat += step
    return grid

In [12]:
def get_target_area(area, region_step, target_step):
    
    regions = generate_grid(area, region_step)
    print('-' * 50)
    print("Cluster: {}".format(cluster))
    print('-' * 50)
    print("THE REGIONS ARE {}".format(regions))
    print('-' * 50)
    
    errors_list = []
    
    for n,region in enumerate(regions):
        boxes = generate_grid(region, target_step)
        print('-' * 50)
        print("REGION #{} OUT OF {} IS: {}".format(n+1, len(regions), region))
        print('-' * 50)
        print("THE BOXES IN REGION #{} ARE {}".format(n+1,boxes))
        print('-' * 50)

        errors = get_box_dask(boxes)
        errors_list.append([errors])
        
        print('-' * 50)
        print("COMPLETED REGION #{} OUT OF {}".format(n+1,len(regions)))
        print('-' * 50)
        
    return errors_list

In [13]:
atl1 = [-75,25,-90,0,0,2000]
atl2 = [-75,25,0,90,0,2000]
ind1 = [25,120,-90,30,0,2000]
pac1 = [120,180,-90,0,0,2000]
pac2 = [120,180,0,90,0,2000]
pac3 = [-180,-75,-90,0,0,2000]
pac4 = [-180,-75,0,90,0,2000]

In [14]:
globe = [-180, 180, -90, 90, 0, 2002]

In [15]:
problem = [-130,-75,-90,10,0,2002]

problem1 = [-125,-75,-25,25,0,2000]
problem2 = [-125,-75,-75,-25,0,2000]
problem3 = [-175,-125,-25,25,0,2000]
problem4 = [-175,-125,-75,-25,0,2000]

Need to rerun boxes[2]!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [15]:
area = atl1
region_step = 50
box_step = 5
interp_step = 2
get_target_area(area, region_step, box_step)

--------------------------------------------------
Cluster: LocalCluster(bbba1d7e, 'tcp://127.0.0.1:35433', workers=32, threads=64, memory=0.94 TiB)
--------------------------------------------------
THE REGIONS ARE [[-90, -40, -75, -25, 0, 2000], [-90, -40, -25, 25, 0, 2000], [-40, 0, -75, -25, 0, 2000], [-40, 0, -25, 25, 0, 2000]]
--------------------------------------------------
--------------------------------------------------
REGION #1 OUT OF 4 IS: [-90, -40, -75, -25, 0, 2000]
--------------------------------------------------
THE BOXES IN REGION #1 ARE [[-75, -70, -90, -85, 0, 2000], [-75, -70, -85, -80, 0, 2000], [-75, -70, -80, -75, 0, 2000], [-75, -70, -75, -70, 0, 2000], [-75, -70, -70, -65, 0, 2000], [-75, -70, -65, -60, 0, 2000], [-75, -70, -60, -55, 0, 2000], [-75, -70, -55, -50, 0, 2000], [-75, -70, -50, -45, 0, 2000], [-75, -70, -45, -40, 0, 2000], [-70, -65, -90, -85, 0, 2000], [-70, -65, -85, -80, 0, 2000], [-70, -65, -80, -75, 0, 2000], [-70, -65, -75, -70, 0, 2000

/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)

KeyboardInterrupt

2025-08-21 14:37:08,841 - distributed.worker - ERROR - 
Traceback (most recent call last):
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/base_events.py", line 678, in run_until_complete
    self.run_forever()
  File "

loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
to xarray complete
point to profile complete
NEW INTERP FUNCTION
interpolation complete
adding spice complete
to xarray complete
point to profile complete
NEW INTERP FUNCTION
interpolation complete
adding spice complete
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
to xarray complete
point to profile complete
NEW INTERP FUNCTION
interpolation complete
add

Process Dask Worker process (from Nanny):
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-08-21 14:37:09,146 - distributed.nanny - ERROR - Worker process died unexpectedly
Proc

loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY
to xarray complete


Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/base_events.py", line 678, in run_until_complete
    self.run_forever()
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/base_events.py", line 645, in run_forever
    self._run_once()
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/lib/python3.12/asyncio/base_events.py", line 1961, in _run_once
    event_list = self._selector.select(timeout)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home.ufs/amf2288/miniforge3/envs/Argo_Feb_25/li

loading points complete
USING LOCAL ARGOPY DIRECTORY
loading points complete
USING LOCAL ARGOPY DIRECTORY


In [1]:
# Shut down the client
client.close()

# Shut down the cluster
cluster.close()


NameError: name 'client' is not defined

# Data Loading Issues

In [ ]:
ds = xr.open_dataset('/swot/SUM05/amf2288/sync-boxes/test_error.nc')

In [ ]:
fig = plt.figure(figsize=(5, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)

ax.scatter(ds.LONGITUDE,ds.LATITUDE,transform=ccrs.PlateCarree(),color='tab:blue',s=1)